In [51]:
import findspark
findspark.init()

import datetime as dt
import pymysql
import pandas.io.sql as psql
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
import pyinputplus



In [30]:
spark = SparkSession.builder.master("local[1]").appName('Credit_card.com').getOrCreate()

In [52]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='creditcard_capstone'
)


1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.

In [53]:
cursor = connection.cursor()

In [47]:
sql = "SELECT c.CUST_ZIP,c.SSN,cc.CUST_SSN,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.TIMEID FROM cdw_sapp_customer c  JOIN cdw_sapp_credit_card cc ON c.SSN=cc.CUST_SSN "
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

C:\Users\Learner_XZHCG225\AppData\Local\Temp\ipykernel_21040\3176533159.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_frame = psql.read_sql(sql, con=connection)


In [42]:
data_frame

,CUST_ZIP,SSN,CUST_SSN,TRANSACTION_VALUE,TRANSACTION_TYPE,TIMEID
0,21042,123456698,123456698,25.49,Entertainment,20181220
1,21042,123456698,123456698,47.08,Grocery,20181221
2,21042,123456698,123456698,82.87,Test,20181101
3,21042,123456698,123456698,65.46,Bills,20180114
4,21042,123456698,123456698,95.63,Grocery,20180808
...,...,...,...,...,...,...
46689,21206,123452196,123452196,96.18,Gas,20180514
46690,21206,123452196,123452196,76.23,Entertainment,20180921
46691,21206,123452196,123452196,39.10,Grocery,20180711
46692,21206,123452196,123452196,73.29,Education,20180317


In [35]:
spark_df = spark.createDataFrame(data_frame)

C:\Spark\python\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [24]:
data_frame["TIMEID"] = pd.to_datetime(data_frame["TIMEID"],format='%Y%m%d')

In [ ]:
data_frame

In [26]:
def tansaction_mod1():
    zipcode = 21042 
    month = 11
    year = 2018
    customer_transaction = data_frame[(data_frame.CUST_ZIP == zipcode) &
                                      (data_frame.TIMEID.dt.month == month) &
                                      (data_frame.TIMEID.dt.year == year)]
    daily_values = customer_transaction.groupby(customer_transaction.TIMEID.dt.day)["TRANSACTION_VALUE"].sum()
    result =pd.DataFrame({"Day": daily_values.index, "Transaction Value": daily_values.values})
    result = result.sort_values("Day", ascending=False)
    print(result)

    

In [28]:
tansaction_mod1()

    Day  Transaction Value
12   28              44.55
11   27             142.71
10   24              70.38
9    21              41.48
8    20             161.81
7    13              61.27
6    12              63.23
5    11              47.01
4     9             185.87
3     6              10.59
2     3             103.98
1     2              81.29
0     1             176.93


In [ ]:
   Used to display the number and total values of transactions for a given type.

In [50]:
def transaction_mod2():
    transaction_type = 'Bills'
    transaction_type = data_frame[(data_frame['TRANSACTION_TYPE']=='Bills')&(data_frame['TRANSACTION_VALUE'])]
    transaction_df = transaction_type.groupby('TRANSACTION_TYPE').agg(trans_count=('TRANSACTION_TYPE', 'count'), 
                                                  trans_val=('TRANSACTION_VALUE', 'sum'))
    transaction_df['trans_val'] = round(transaction_df['trans_val'], 2)
    print(transaction_df)

transaction_mod2()



                  trans_count  trans_val
TRANSACTION_TYPE                        
Bills                    6861  351405.28


Used to display the total number and total values of transactions for branches in a given state.

In [54]:
sql = "SELECT bc.BRANCH_CODE,bc.BRANCH_STATE,cc.TRANSACTION_VALUE,cc.TRANSACTION_TYPE,cc.BRANCH_CODE FROM cdw_sapp_branch bc JOIN cdw_sapp_credit_card cc ON bc.BRANCH_CODE=cc.BRANCH_CODE"
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

C:\Users\Learner_XZHCG225\AppData\Local\Temp\ipykernel_21040\914311548.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_frame = psql.read_sql(sql, con=connection)


In [55]:
data_frame

,BRANCH_CODE,BRANCH_STATE,TRANSACTION_VALUE,TRANSACTION_TYPE,BRANCH_CODE
0,43,TX,25.49,Entertainment,43
1,63,MS,47.08,Grocery,63
2,12,MI,82.87,Test,12
3,44,NC,65.46,Bills,44
4,7,NJ,95.63,Grocery,7
...,...,...,...,...,...
46689,72,OH,96.18,Gas,72
46690,42,OH,76.23,Entertainment,42
46691,107,OH,39.10,Grocery,107
46692,15,PA,73.29,Education,15


In [61]:
def transaction_mod3():
        branch_state = 'PA'
        branch_transaction = data_frame[(data_frame['BRANCH_STATE']=='PA')&(data_frame['TRANSACTION_VALUE'])]
        trans_by_branch_state = branch_transaction.groupby([ 'BRANCH_STATE']).agg(total_transactions=('TRANSACTION_VALUE', 'sum'))
        print(trans_by_branch_state)

transaction_mod3()

              total_transactions
BRANCH_STATE                    
PA                     227451.26
